In [ ]:
%matplotlib inline

In [ ]:
import gc
import pathlib
import sys

In [ ]:
import joblib
import lightgbm as lgb
import pandas as pd

In [ ]:
src_dir = "../../src"

In [ ]:
sys.path.append(src_dir)

In [ ]:
from package.constants import *
from package.utils import *

In [ ]:
lgb_params["metric"] = "binary_logloss"
lgb_params["objective"] = "binary"

In [ ]:
train = pd.read_parquet(train_path)

In [ ]:
# See https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/135896
is_train = (train["date"] >= train_start_date) & (train["date"] < validation_start_date)
is_valid = (train["date"] >= validation_start_date) & (train["date"] < test_start_date)

In [ ]:
is_train |= is_valid
# See https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/138268
is_train &= train["sell_price"].notnull()

In [ ]:
%%time
train["is_bought"] = train[transformed_target] > 0.0
# See https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/149754
dtrain = create_dataset(
    train, is_train, features, "is_bought", categorical_feature=categorical_features
)

In [ ]:
del train
del is_train
del is_valid

In [ ]:
gc.collect()

In [ ]:
%%time
model = lgb.train(lgb_params, dtrain, valid_sets=[dtrain], verbose_eval=10)

In [ ]:
ax = lgb.plot_importance(model, importance_type="split", figsize=(16, 9))

In [ ]:
ax = lgb.plot_importance(model, importance_type="gain", figsize=(16, 9))

In [ ]:
joblib.dump(model, lgbm_clf_path)